In [2]:
import gdal
from glob import glob

In [3]:
def add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'

In [4]:
def convert_rasters(src_dir, dest_dir, epsg_format, windows=True):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""

    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    input_files = glob(src_dir + '*/*.jp2')
    # Keep track of how many files were converted
    n = 1
    total = len(input_files)
    
    for f in input_files:
        print(f'processing file {n} of {total}')
        n += 1
        
        # The way we've set it up, we save each product into a numbered folder,
        # depending on which layer it's in. To keep this structure, we need to
        # pull out the folder number from the file path.
        # How exactly to do this depends on if you're using Windows or not,
        # since the path conventions are different.
        if windows:
            folder_num = f.split('\\')[-2]
            filename = f.split('\\')[-1]
        else:
            folder_num = f.split('/')[-2]
            filename = f.split('/')[-1]
        output_folder = dest_dir + folder_num + '/'
        
        # If the output folder doesn't exist, create it
        if not os.path.isdir(output_folder):
            os.mkdir(output_folder)
        
        output_filepath = output_folder + filename

        # Finally, we convert
        converted = gdal.Warp(output_filepath, f,
                              dstSRS=epsg_format, resampleAlg='near', format='GTiff')
        converted = None
        
    print('Finished')

In [5]:
def make_full_virtual_raster(src_dir, dest_dir, num_layers=10):
    """Combines the rasters in the src_dir into a single virtual raster
    with proper prioritization. This is saved into the dest_dir.
    Make sure the num_layers variable is the same as the number of tile layers
    in your src_dir."""
    
    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    for layer in range(1, num_layers+1):
        print('Making Layer', layer)
        
        # Get the filenames from the layer in question
        filenames = glob.glob(src_dir + f'{layer}/*.jp2', recursive=True)
        
        output_file = dest_dir + f'Layer{layer}.vrt'
    
        vrt = gdal.BuildVRT(output_file, filenames, resolution='average', resampleAlg='nearest', srcNodata=0)
    
        vrt.FlushCache()
    
    print('Making full raster')

    # To make the full raster, we combine every layer. Do it in reverse order because (I believe)
    # the last items in the list are prioritized.

    input_files = [dest_dir + f'Layer{i}.vrt' for i in reversed(range(1, num_layers+1))]
    
    output_file = dest_dir + 'full.vrt'

    vrt = gdal.BuildVRT(output_file, input_files, resolution='average', resampleAlg='nearest', srcNodata=0)

    vrt.FlushCache()

    print('Finished')